In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import numpy as np
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

2022-11-13 14:42:20.277034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
def get_plain_and_debaised_model():
    dataset_orig= load_preproc_data_adult()
    return dataset_orig
    dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

    privileged_groups = [{'sex': 1}]
    unprivileged_groups = [{'sex': 0}]

    sess_plain = tf.Session()
    plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess_plain)

    plain_model.fit(dataset_orig_train)

    tf.reset_default_graph()
    sess_debiased = tf.Session()
    debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess_debiased)

    debiased_model.fit(dataset_orig_train)

    return plain_model, debiased_model


# plain_model, debiased_model = get_plain_and_debaised_model()
dataset_orig = get_plain_and_debaised_model()

In [12]:
# print(dataset_orig)
dataset_orig_df = dataset_orig.convert_to_dataframe()[0] # convert to pandas dataframe
dataset_orig_df.head()

,race,sex,Age (decade)=10,Age (decade)=20,Age (decade)=30,Age (decade)=40,Age (decade)=50,Age (decade)=60,Age (decade)=>=70,Education Years=6,Education Years=7,Education Years=8,Education Years=9,Education Years=10,Education Years=11,Education Years=12,Education Years=<6,Education Years=>12,Income Binary
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def create_single_entry_adult(race, sex, age, education_years):
    arr = np.array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

    arr[0][0] = race
    arr[0][1] = sex
    if age <= 19:
        arr[0][2] = 1.
    elif age <= 29:
        arr[0][3] = 1.
    elif age <= 39:
        arr[0][4] = 1.
    elif age <= 49:
        arr[0][5] = 1.
    elif age <= 59:
        arr[0][6] = 1.
    elif age <= 69:
        arr[0][7] = 1.
    else:
        arr[0][8] = 1.

    if education_years < 6:
        arr[0][16] = 1.
    elif education_years == 6:
        arr[0][9] = 1.
    elif education_years == 7:
        arr[0][10] = 1.
    elif education_years == 8:
        arr[0][11] = 1.
    elif education_years == 9:
        arr[0][12] = 1.
    elif education_years == 10:
        arr[0][13] = 1.
    elif education_years == 11:
        arr[0][14] = 1.
    elif education_years == 12:
        arr[0][15] = 1.
    else:
        arr[0][17] = 1.
    
    dataset_replaced_data = load_preproc_data_adult()
    dataset_replaced_data.features = arr
    dataset_replaced_data.age = age
    dataset_replaced_data.edu = education_years
    return dataset_replaced_data

# IDK what the race and sex is - that is, idk what sex = 1 means or race = 0
# not sure where to find that
example_input = create_single_entry_adult(1, 0, 65, 13)
# we can make a script that prompts a user to input sex, race, age, and education and input to this^^^

In [9]:
def predict_single_income(model, user_input):
    pred = model.predict(user_input).labels[0][0]
    
    races = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black']
    genders = ['Female', 'Male']

    race_print = f"(race={races[int(user_input.features[0][0])]})" # idk which value is which race so this is the placeholder 
    sex_print = f"(sex={genders[int(user_input.features[0][1])]})" # idk which value is which sex so this is the placeholder 

    if pred == 1.0:
        print(f"The model predicts that a {user_input.age} year old {race_print} {sex_print} with {user_input.edu} years of education DOES have an income greater than 50k.")
    elif pred == 0.0:
        print(f"The model predicts that a {user_input.age} year old {race_print} {sex_print} with {user_input.edu} years of education DOES NOT have an income greater than 50k.")

predict_single_income(plain_model, example_input)

The model predicts that a 65 year old (race=Asian-Pac-Islander) (sex=Female) with 13 years of education DOES NOT have an income greater than 50k.


In [10]:
input1 = create_single_entry_adult(1, 1, 65, 13)
input2 = create_single_entry_adult(1, 0, 65, 13)

predict_single_income(plain_model, input1)
predict_single_income(plain_model, input2)

The model predicts that a 65 year old (race=Asian-Pac-Islander) (sex=Male) with 13 years of education DOES have an income greater than 50k.
The model predicts that a 65 year old (race=Asian-Pac-Islander) (sex=Female) with 13 years of education DOES NOT have an income greater than 50k.


In [11]:
input1 = create_single_entry_adult(1, 1, 65, 11)
input2 = create_single_entry_adult(1, 0, 65, 11)

predict_single_income(debiased_model, input1)
predict_single_income(debiased_model, input2)

The model predicts that a 65 year old (race=Asian-Pac-Islander) (sex=Male) with 11 years of education DOES NOT have an income greater than 50k.
The model predicts that a 65 year old (race=Asian-Pac-Islander) (sex=Female) with 11 years of education DOES NOT have an income greater than 50k.
